# Import library


In [95]:
import torch
import math
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision

from sklearn.model_selection import train_test_split
from typing import Any
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Setup device and tensorboard


In [96]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

# Customized Dataset


In [97]:
class ImageDataset(Dataset):
    def __init__(self, list_root_dir: list, transform=None) -> None:
        super().__init__()
        self.list_root_dir = list_root_dir
        self.transform = transform

        self.image_paths = []
        for root_dir in list_root_dir:
            self.image_paths.extend(
                glob.glob(os.path.join(root_dir, "with", "*.jpg")))
            self.image_paths.extend(
                glob.glob(os.path.join(root_dir, "without", "*.jpg")))
        random.shuffle(self.image_paths)

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, index) -> Any:
        image_path = self.image_paths[index]

        label = 1 if image_path.split(os.sep)[-2] == "with" else 0
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return [image, label]

# Train one epoch


In [98]:
def train_model(model, loss_function, optimizer, train_loader, writer, epoch=1):
    model.train()
    running_loss = 0.0
    total_samples = 0
    y_true = []
    y_pred = []
    for data in train_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # Clear gradient
        optimizer.zero_grad()
        # Calculate logits
        outputs = model(images)
        # Calculate loss
        loss = loss_function(outputs, labels)
        # Calculate gradient from loss
        loss.backward()
        # Update weight
        optimizer.step()

        # Calculate loss
        running_loss += loss.item() * images.size(0)
        # Calculate total sample in data_loader
        total_samples += images.size(0)
        # Calculte y_predict for evaluation
        predicted = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.detach().cpu().numpy())
    loss = running_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("train/loss", loss, epoch)
    writer.add_scalar("train/accuracy", accuracy, epoch)
    return loss, accuracy

# Eval model


In [99]:
def eval_model(model, loss_function, test_dataloader, writer, epoch):
    model.eval()
    total_samples = 0
    total_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Calculate logits
            outputs = model(images)

            # Calculate loss of outputs and y_true
            loss = loss_function(outputs, labels)
            total_loss += loss.item() * images.size(0)
            # Calculate total sample
            total_samples += images.size(0)

            # Calculte y_predict for evaluation
            predicted = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.detach().cpu().numpy())
    loss = total_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("test/loss", loss, epoch)
    writer.add_scalar("test/accuracy", accuracy, epoch)
    return loss, accuracy

# Model


In [100]:
model = torchvision.models.resnet34(num_classes=2)
model = model.to(device)
print(device)

cuda:0


# Optimizer


In [101]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Loss function


In [102]:
loss_function = torch.nn.CrossEntropyLoss()

# Train


In [104]:
model = torchvision.models.mobilenet_v3_large(num_classes=2)
model = model.to(device)
model_name = "mobilenet_v3_large"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

epoch 0: train_loss 0.6923, train_acc 0.5311, test_loss 0.6929330935861773, test_acc 0.5814
epoch 1: train_loss 0.6924, train_acc 0.53, test_loss 0.6928534517558445, test_acc 0.5814
epoch 2: train_loss 0.6923, train_acc 0.5308, test_loss 0.6928335264788249, test_acc 0.5814
epoch 3: train_loss 0.6923, train_acc 0.5252, test_loss 0.6929117808394284, test_acc 0.5804
epoch 4: train_loss 0.6922, train_acc 0.5396, test_loss 0.6928274711045748, test_acc 0.5137
epoch 5: train_loss 0.6922, train_acc 0.5326, test_loss 0.6907804782691341, test_acc 0.5192
epoch 6: train_loss 0.6922, train_acc 0.5259, test_loss 0.6907749721947274, test_acc 0.5338
epoch 7: train_loss 0.6923, train_acc 0.5289, test_loss 0.6901662920685093, test_acc 0.5356
epoch 8: train_loss 0.6922, train_acc 0.5294, test_loss 0.6903439388632556, test_acc 0.543
epoch 9: train_loss 0.6923, train_acc 0.5218, test_loss 0.6903235368580444, test_acc 0.5448
epoch 10: train_loss 0.6923, train_acc 0.5299, test_loss 0.6901334370511977, test_a

In [ ]:
model = torchvision.models.efficientnet_b1(num_classes=2)
model = model.to(device)
model_name = "efficientnet_b1"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

epoch 0: train_loss 2.6591, train_acc 0.4917, test_loss 1.5725709575839626, test_acc 0.5311
epoch 1: train_loss 2.6633, train_acc 0.4836, test_loss 2.7813341988070137, test_acc 0.5265
epoch 2: train_loss 2.7235, train_acc 0.4821, test_loss 2.6274256297613627, test_acc 0.5393
epoch 3: train_loss 2.6632, train_acc 0.4865, test_loss 3.1331885618309214, test_acc 0.5302
epoch 4: train_loss 2.6484, train_acc 0.4814, test_loss 4.397794828118529, test_acc 0.5137
epoch 5: train_loss 2.6105, train_acc 0.4858, test_loss 4.400562539288069, test_acc 0.5366
epoch 6: train_loss 2.7231, train_acc 0.4862, test_loss 3.076007527014891, test_acc 0.5302
epoch 7: train_loss 2.5961, train_acc 0.4939, test_loss 2.751560049257505, test_acc 0.5293
epoch 8: train_loss 2.651, train_acc 0.4892, test_loss 4.243332407810135, test_acc 0.5384
epoch 9: train_loss 2.6727, train_acc 0.4897, test_loss 4.146779999000716, test_acc 0.5247
epoch 10: train_loss 2.6123, train_acc 0.4876, test_loss 4.379474850633681, test_acc 0.

In [ ]:
model = torchvision.models.efficientnet_b2(num_classes=2)
model = model.to(device)
model_name = "efficientnet_b2"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

epoch 0: train_loss 2.3955, train_acc 0.4445, test_loss 1.3501771601288148, test_acc 0.4488
epoch 1: train_loss 2.3438, train_acc 0.4505, test_loss 1.2039489236130794, test_acc 0.4488
epoch 2: train_loss 2.3009, train_acc 0.4451, test_loss 1.2711152300320334, test_acc 0.4534
epoch 3: train_loss 2.3256, train_acc 0.4511, test_loss 1.4797884856761048, test_acc 0.4442
epoch 4: train_loss 2.3433, train_acc 0.4389, test_loss 1.2405534029224892, test_acc 0.4543
epoch 5: train_loss 2.3147, train_acc 0.4441, test_loss 1.6985297714116587, test_acc 0.4442
epoch 6: train_loss 2.3167, train_acc 0.4464, test_loss 1.4024876534829847, test_acc 0.4479
epoch 7: train_loss 2.2967, train_acc 0.453, test_loss 1.2248357270709775, test_acc 0.4424
epoch 8: train_loss 2.3363, train_acc 0.4444, test_loss 1.3597585704906332, test_acc 0.4497
epoch 9: train_loss 2.2818, train_acc 0.4383, test_loss 1.46485254380995, test_acc 0.4525
epoch 10: train_loss 2.3191, train_acc 0.4525, test_loss 1.246889396276073, test_ac

In [ ]:
model = torchvision.models.efficientnet_b3(num_classes=2)
model = model.to(device)
model_name = "efficientnet_b3"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

epoch 0: train_loss 3.6635, train_acc 0.4995, test_loss 2.9989221259488485, test_acc 0.5174
epoch 1: train_loss 3.7573, train_acc 0.5006, test_loss 3.803492504565328, test_acc 0.5146
epoch 2: train_loss 3.899, train_acc 0.4955, test_loss 2.5495735003699767, test_acc 0.5311
epoch 3: train_loss 3.7823, train_acc 0.4976, test_loss 2.6692805422071566, test_acc 0.5256
epoch 4: train_loss 3.798, train_acc 0.501, test_loss 2.501239156483296, test_acc 0.5183
epoch 5: train_loss 3.9068, train_acc 0.4974, test_loss 2.651037236018416, test_acc 0.5229
epoch 6: train_loss 3.7092, train_acc 0.5041, test_loss 2.6691781942761574, test_acc 0.5311
epoch 7: train_loss 3.649, train_acc 0.5097, test_loss 2.539135395935728, test_acc 0.5201
epoch 8: train_loss 3.8652, train_acc 0.4986, test_loss 3.1363655784230344, test_acc 0.5165
epoch 9: train_loss 3.8377, train_acc 0.499, test_loss 2.200179038906446, test_acc 0.5027
epoch 10: train_loss 3.8121, train_acc 0.4958, test_loss 1.989823589813121, test_acc 0.531

In [ ]:
model = torchvision.models.efficientnet_b4(num_classes=2)
model = model.to(device)
model_name = "efficientnet_b4"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

epoch 0: train_loss 2.1891, train_acc 0.5143, test_loss 2.9527932673529254, test_acc 0.4717
epoch 1: train_loss 2.1638, train_acc 0.5105, test_loss 2.9602517661054564, test_acc 0.4771
epoch 2: train_loss 2.2029, train_acc 0.5091, test_loss 4.062865914112907, test_acc 0.4698
epoch 3: train_loss 2.2607, train_acc 0.507, test_loss 2.9801703365672876, test_acc 0.457
epoch 4: train_loss 2.3721, train_acc 0.4998, test_loss 4.0095533298182096, test_acc 0.468
epoch 5: train_loss 2.2555, train_acc 0.51, test_loss 2.656771469813596, test_acc 0.468
epoch 6: train_loss 2.3189, train_acc 0.5118, test_loss 4.672930087969116, test_acc 0.4717
epoch 7: train_loss 2.3025, train_acc 0.5009, test_loss 4.0043778398791, test_acc 0.4561
epoch 8: train_loss 2.2065, train_acc 0.5095, test_loss 3.2471170891791417, test_acc 0.4698
epoch 9: train_loss 2.2769, train_acc 0.5069, test_loss 2.461978888816764, test_acc 0.4625
epoch 10: train_loss 2.4345, train_acc 0.51, test_loss 2.8946651595166224, test_acc 0.4589
ep

In [ ]:
model = torchvision.models.efficientnet_v2_s(num_classes=2)
model = model.to(device)
model_name = "efficientnet_v2_s"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

KeyboardInterrupt: 

In [ ]:
model = torchvision.models.efficientnet_v2_m(num_classes=2)
model = model.to(device)
model_name = "efficientnet_v2_m"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

In [ ]:
model = torchvision.models.efficientnet_v2_l(num_classes=2)
model = model.to(device)
model_name = "efficientnet_v2_l"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()